
1) Dado los acontecimientos en USA, deseamos obtener datos que nos den mayor
información sobre las muertes de personas de raza negra por parte de oficiales de policía.
Para ello, tenemos un csv con información sobre las muertes por parte de oficiales
de policía en USA desde 2015 hasta 2017:

(name, date, race, city, state)

Y otro csv con información sobre el porcentaje de cada raza en las ciudades de USA:

(state, city, share_white, share_black, share_native_american, share_asian, share_hispanic)

Se pide:
a) Obtener el estado con mayor porcentaje de muertes de personas de raza negra
teniendo en cuenta la cantidad total de muertes por parte de oficiales en ese estado. (10 pts)

b) Obtener los 10 estados con mayor diferencia entre el porcentaje de muertes y
el porcentaje de gente de raza negra en ese estado. Para ello, considerar el
porcentaje de raza de un estado como el promedio de los valores de sus ciudades.
Por ejemplo si en Texas el porcentaje de muertes de personas de raza negra por
parte de la policía es del 36% y el promedio de share_black para Texas es 24%
la diferencia es 0.12. (15 pts)

Resolver ambos puntos usando la API de RDDs de PySpark.

In [3]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

murdersRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-1Op.csv', header=True).rdd
'''
def seq_op(acc, elem):
    if elem == 'Black':
        return acc[0] + 1, acc[1] + 1
    return acc[0], acc[1] + 1

comb_op = lambda acc1,acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])

blackMurderRate = murdersRdd.map(lambda x: (x[4], x[2])).aggregateByKey((0, 0), seq_op, comb_op) \
            .mapValues(lambda x: x[0]/x[1]).cache()
''' # esta es una forma de hacerlo con aggregate

blackMurderRate = murdersRdd.map(lambda x: (x[4], (1 if (x[2] == 'Black') else 0, 1))) \
            .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0]/x[1]).cache()

blackMurderRate.reduce(lambda x,y: x if (x[1] > y[1]) else y)

('S2', 0.2066869300911854)

In [4]:
statsCitiesRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-1Opb.csv', header=True).rdd

blackSharePerState = statsCitiesRdd.map(lambda x: (x[0], (pd.to_numeric(x[3], errors='coerce'), 1)))\
            .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0] / x[1])

blackMurderRate.join(blackSharePerState).mapValues(lambda x: x[0] - x[1]).takeOrdered(10, key=lambda x: x[1])

[('S1', -0.2), ('S2', -0.09331306990881458), ('S3', -0.06071428571428575)]